# 6. Korpuszépítés

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## Crawling és scraping

robots.txt

sitemap.xml

`wget`: parancssoros letöltő program

* `-m` vagy `--mirror`: teljes oldal (site) tükrözése, ez minden lapot (page) letölt
* `-r` vagy `--recursive`: a megadott lapon található linkeket is letölti + az azokon találhatóakat is, sít.
* `-l depth`: a rekuzív bejárás mélysége adható meg ezzel (default 5)
* `-nd`: csak a fájlokat tölti le, nem hozza létre a könyvtárszerkezetet
* `-P path/to/dir/`: célkönyvtár megadása
* `-A acclist` ill. `-R rejlist`: elfogadandó és elutasítandó suffixek listája

In [ ]:
%%bash

#rm -rf guttenberg/
#mkdir guttenberg
#wget -r -l 2 -P guttenberg/ -nd -q -A ".txt" http://www.gutenberg.org/ebooks/author/5845